In [12]:
import numpy as np
import pandas as pd
import csv
from matplotlib import pyplot as plt
from sklearn.cluster import MeanShift
%matplotlib inline

In [3]:
with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [4]:
data = pd.read_csv('checkins.csv')

In [5]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [9]:
data1 = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
data1.dropna(inplace=True)
data1.head()

/home/sid/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [10]:
data.shape

(396634, 6)

In [20]:
cluster = MeanShift(bandwidth=0.1)
X = data.values[0:100000, 3:5]

In [21]:
%%time
cluster.fit(X)

CPU times: user 2min 6s, sys: 0 ns, total: 2min 6s
Wall time: 2min 6s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [22]:
cluster.labels_

array([ 5,  7, 30, ..., 25, 19,  4])

In [24]:
centers = pd.read_csv('centers.csv')
centers.head()

,latitude,longitude,city
0,33.751277,-118.188740,Los Angeles
1,25.867736,-80.324116,Miami
2,51.503016,-0.075479,London
3,52.378894,4.885084,Amsterdam
4,39.366487,117.036146,Beijing


In [25]:
nearest = cluster.predict(centers.drop('city', axis=1).values)

In [26]:
nearest

array([  51,  419,   58,  370, 1980,  420])

In [27]:
labels = cluster.labels_
cluster_centers = cluster.cluster_centers_
n_clusters = len(labels)

In [28]:
clusters_select = np.ndarray(shape=(593,2))

In [29]:
clusters_select

array([[6.90922194e-310, 6.90922194e-310],
       [4.64354512e-310, 4.64354512e-310],
       [1.64953213e+150, 2.46155016e+179],
       ...,
       [5.98147383e-154, 4.65203811e+151],
       [5.81846707e+180, 6.21063012e+175],
       [6.01334434e-154, 3.09252835e+262]])

In [30]:
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] += 1

In [31]:
i = 0
j = 0
while i < len(cluster_centers):
    if d[i] > 15:
        clusters_select[j] = cluster_centers[i]
        j += 1
    i += 1

In [32]:
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [34]:
answer_index = 0
min_dist = 0
i = 0
while i < len(clusters_select):
    distances = [distance(xx, clusters_select[i]) for xx in centers.drop('city', axis=1).values]
    if min_dist == 0:
        min_dist = min(distances)
        answer_index = i
    else:
        if min_dist > min(distances):
            min_dist = min(distances)
            answer_index = i
    i += 1

/home/sid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  


In [35]:
answer_index

417

array([  33.751277, -118.18874 ])